In [ ]:
!pip install "google-cloud-aiplatform>=1.38"

In [ ]:
!pip install pillow
!pip install matplotlib

In [2]:
import os
import vertexai
from vertexai.language_models import TextGenerationModel

In [4]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "./classhub-411006-522c3827b5bb.json"

# 리뷰 텍스트 전처리

In [ ]:
!pip install pymysql
!pip install sqlalchemy

In [15]:
import pandas as pd
import pymysql
from sqlalchemy import create_engine
import json
from sqlalchemy.orm import sessionmaker
import json

In [48]:
try:
    # MySQL 설정값
    host = 'i10a810.p.ssafy.io'
    port = 3306
    user = 'root'  # 데이터베이스 사용자 이름을 입력해주세요.
    password = 'ssafya810'  # 데이터베이스 비밀번호를 입력해주세요.
    db_name = 'classhub'
    # SQLAlchemy 엔진 객체 생성
    engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{db_name}")

    # Session 생성
    Session = sessionmaker(bind=engine)
    session = Session()
except:
    raise Exception("세션 연결 오류")

review_df = pd.read_sql('''SELECT sr.*, l.site_lecture_id
            FROM site_review sr
            inner join lecture l
            on sr.lecture_id=l.lecture_id
            where content!=''
            ''', engine)


In [49]:
review_df["site_lecture_id"] = review_df["site_lecture_id"].apply(lambda x:int(x[1:]))
review_df

,site_review_id,score,content,lecture_id,update_time,create_time,site_lecture_id
0,7703289,5.0,The way Mashrur explains everything it's great...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010
1,7703292,5.0,Thanks Mashrur for detailed and hands-on course.,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010
2,7703311,5.0,I've learn a lot from this course. It's very h...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010
3,7703319,4.5,"Todo el curso estuvo muy bien presentado, buen...",20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010
4,7703329,4.0,It covers the details related to Rails convent...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010
...,...,...,...,...,...,...,...
1803472,2572882,4.0,The narration is clear and to the point,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928
1803473,2572886,5.0,A well done and unique perspective on programm...,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928
1803474,2572888,5.0,I like the way the course is made.,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928
1803475,2572889,5.0,I am a beginner and the tutor invested her tim...,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928


In [50]:
review_df.to_csv("./raw_reviews.csv",index=False)

In [34]:
"12 34".split()

['12', '34']

In [28]:
review_df = pd.read_csv("./raw_reviews.csv")

In [51]:
review_df
review_df["status"] = review_df["content"].apply(lambda x:True if (len(x)>=5 and len(x.split())>=2) else False)
review_df = review_df[review_df["status"]]

In [52]:
net_df = review_df.groupby('site_lecture_id').size().reset_index(name='count').sort_values(by='count',ascending=False)
net_df = net_df[net_df["count"]>=5]
net_df
# review_df.groupby('lecture_id').size().sort_values(by='content',ascending=False)

,site_lecture_id,count
3785,987106,3043
3044,854544,1500
11161,1927558,1233
18730,2846906,1116
3567,948840,1010
...,...,...
27656,4014252,5
27653,4014214,5
37500,4852688,5
26741,3916956,5


In [30]:
review_df

,site_review_id,score,content,lecture_id,update_time,create_time,status
2,42,3.5,It is a good sumup of the possibilities for th...,15966,2024-02-08 05:33:02,2024-02-08 05:33:02,True
3,44,5.0,the contents are easy to learn and interactive,15966,2024-02-08 05:33:02,2024-02-08 05:33:02,True
4,45,5.0,"Simple, concise and easy to follow for a 2 ho...",15966,2024-02-08 05:33:02,2024-02-08 05:33:02,True
5,63,4.0,It's fascinating,15966,2024-02-08 05:33:02,2024-02-08 05:33:02,True
6,71,4.0,This is an interesting course showing the actu...,15966,2024-02-08 05:33:02,2024-02-08 05:33:02,True
...,...,...,...,...,...,...,...
1803472,9459951,5.0,Great IT Professional having with the best kno...,43585,2024-02-08 05:37:09,2024-02-08 05:37:09,True
1803473,9459953,4.5,How about you run the text in the course for s...,43585,2024-02-08 05:37:09,2024-02-08 05:37:09,True
1803474,9459956,5.0,Questions are framed according to the certific...,43585,2024-02-08 05:37:09,2024-02-08 05:37:09,True
1803475,9459958,5.0,Great question bank for google cloud certifica...,43585,2024-02-08 05:37:09,2024-02-08 05:37:09,True


In [14]:
20*60*60*200/60

240000.0

In [54]:
# 한글 영어만 대상
with open("../korean-weight/ko_list.json", "r", encoding="utf-8") as f:
    ko_list = list(json.load(f))
    
ko_df = pd.DataFrame({
    "site_lecture_id" : ko_list
})

net_df = pd.merge(net_df, ko_df, how="inner", left_on="site_lecture_id", right_on="site_lecture_id")

In [56]:
temp_reviews = pd.merge(review_df, net_df, how="inner", left_on="site_lecture_id", right_on="site_lecture_id")
temp_reviews

,site_review_id,score,content,lecture_id,update_time,create_time,site_lecture_id,status,count
0,7703289,5.0,The way Mashrur explains everything it's great...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010,True,167
1,7703292,5.0,Thanks Mashrur for detailed and hands-on course.,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010,True,167
2,7703311,5.0,I've learn a lot from this course. It's very h...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010,True,167
3,7703319,4.5,"Todo el curso estuvo muy bien presentado, buen...",20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010,True,167
4,7703329,4.0,It covers the details related to Rails convent...,20765,2024-02-08 05:36:18,2024-02-08 05:36:18,1000010,True,167
...,...,...,...,...,...,...,...,...,...
1041213,2572882,4.0,The narration is clear and to the point,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928,True,31
1041214,2572886,5.0,A well done and unique perspective on programm...,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928,True,31
1041215,2572888,5.0,I like the way the course is made.,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928,True,31
1041216,2572889,5.0,I am a beginner and the tutor invested her tim...,33404,2024-02-08 05:33:59,2024-02-08 05:33:59,999928,True,31


In [57]:
joined_df = temp_reviews.groupby('site_lecture_id')["content"].apply(lambda x: '\n'.join(x)[:3600]).reset_index()
joined_df

,site_lecture_id,content
0,8082,To gain knowledge and new skill for personal i...
1,8319,Great and interesting course. Enjoyed every to...
2,8416,Dated content - XCode 4 vs XCode 8 ...\nthis i...
3,9463,The quality is the same as on desktop it is ju...
4,9646,"Excelente curso muy completo, cubre desde lo b..."
...,...,...
22905,5744038,This course is valuable and informative course...
22906,5744808,Thanks a lot. Highly recommended.\nHighly reco...
22907,5744908,I expected more. The course is basically the i...
22908,5745318,Thanks for the detailed explanation on your te...


In [58]:
joined_df["prompt"] = joined_df["content"].apply(lambda x:"지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 요약을 각각 50자 이내로 생성해줘. Please write in korean\n"+x)

In [60]:
joined_df.to_csv("./prompt.csv",index=False)

In [75]:
sum(joined_df[""].apply(lambda x:len(x)).values)/48147

3625.001495420275

In [85]:
prompt_df = pd.read_csv("./prompt.csv")
prompt_df["prompt"] = joined_df["content"].apply(lambda x:"지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 요약을 각각 50자 이내로 생성해줘. Please write in korean\n"+x[:3600])

In [89]:
prompt_df.to_csv("./prompt.csv",index=False)

In [87]:
sum(prompt_df["prompt"].apply(lambda x:len(x)).values)/48147

1692.0782395580202

In [76]:
joined_df["prompt"] = joined_df["content"].apply(lambda x:"지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 요약을 각각 50자 이내로 생성해줘. Please write in korean\n"+x)

In [82]:
joined_df.drop(columns=["good_prompt","bad_prompt"]).to_csv("./prompt.csv",index=False)

In [68]:
location = "us-central1"
project_id = "classhub-411006"
vertexai.init(project=project_id, location=location)

parameters = {
    "max_output_tokens": 256,  # Token limit determines the maximum amount of text output.
}
model = TextGenerationModel.from_pretrained("text-bison@001")

In [77]:
gptReviews = []
for i,row in joined_df.iterrows():
    if(i==6):
        break
    review = {}
    
    response = model.predict(
        row["prompt"],
        **parameters,
    )
    review["lecture_id"] = row["lecture_id"]
    review["gptReviews"] = response.text
    gptReviews.append(review)

In [81]:
pd.DataFrame(gptReviews)

,lecture_id,gptReviews
0,25,강의의 장점:\n- 강의 내용이 잘 정리되어 있다.\n- 강의 진행이 쉽고 이해하...
1,26,장점:\n- 예시가 많아 자세히 설명해준다.\n- 유익하다.\n- 예시를 제공해줬으...
2,28,\nشكرا جزيلا على هذا الكورس الرائع\nشكرا جزيلا...
3,29,\n Good course for beginners.\nThe course is g...
4,30,"장점: 흥미로운 강의, 잘 구성된 자료, 재미있는 강의\n단점: 처음에는 너무 단순..."
5,31,for sharing. 장점:\n- 기초적인 내용을 다루고 있어 초보자가 이해하기 ...


In [84]:
prompt_df = pd.read_csv("./prompt.csv")
prompt_df

,lecture_id,content,length,prompt
0,25,"bad sound, not so good english, too low resolu...",1214,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
1,26,Detailed explanation with plenty of examples.\...,728,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
2,28,الحصص مرتبة بشكل يسهل حضورها وفهمها والشرح كاف...,11258,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
3,29,muy buen curso\nBest course!\nExcellente cours...,10074,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
4,30,"very interesting course, sounds too simple and...",382,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
...,...,...,...,...
48142,59531,Muito boa a didatica.\nFalta atualizar os recu...,17571,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
48143,59533,잘 몰랐던 git 관리에 대해 알 수 있어서 좋았다.,163,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
48144,59534,I started my own project after watching this c...,2430,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
48145,59535,Material is a little old\nThis course is well ...,1038,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...


In [92]:
1000/74*46000/60/60

172.6726726726727

# 다시다시

In [2]:
prompt_df = pd.read_csv("./prompt.csv")
prompt_df

,site_lecture_id,content,prompt
0,8082,To gain knowledge and new skill for personal i...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
1,8319,Great and interesting course. Enjoyed every to...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
2,8416,Dated content - XCode 4 vs XCode 8 ...\nthis i...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
3,9463,The quality is the same as on desktop it is ju...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
4,9646,"Excelente curso muy completo, cubre desde lo b...",지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
...,...,...,...
22905,5744038,This course is valuable and informative course...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
22906,5744808,Thanks a lot. Highly recommended.\nHighly reco...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
22907,5744908,I expected more. The course is basically the i...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...
22908,5745318,Thanks for the detailed explanation on your te...,지시문 : 다음은 하나의 it 인터넷 강의의 리뷰들입니다. 장점과 단점에 해당하는 ...


In [19]:
len("12345")

5

In [12]:
prompt_df["good_prompt"] = prompt_df["content"].apply(lambda x:"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해서 {긍정적인평가} 안에 담아서 제공해줘. Translate all answers into Korean language. \n텍스트들: "+x)
prompt_df["bad_prompt"] = prompt_df["content"].apply(lambda x:"다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 요약해줘. 서론 빼고 문장만 제공해줘. Translate all answers into Korean language. \n텍스트들: "+x)
prompt_df

,site_lecture_id,content,prompt,good_prompt,bad_prompt
0,8082,To gain knowledge and new skill for personal i...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
1,8319,Great and interesting course. Enjoyed every to...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
2,8416,Dated content - XCode 4 vs XCode 8 ...\nthis i...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
3,9463,The quality is the same as on desktop it is ju...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
4,9646,"Excelente curso muy completo, cubre desde lo b...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
...,...,...,...,...,...
22905,5744038,This course is valuable and informative course...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
22906,5744808,Thanks a lot. Highly recommended.\nHighly reco...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
22907,5744908,I expected more. The course is basically the i...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."
22908,5745318,Thanks for the detailed explanation on your te...,"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 두 문장 이내로 ...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해...","다음 텍스트들은 하나의 강의에 대한 평가들이며, 부정적인 평가들을 두 문장 이내로 ..."


In [13]:
prompt_df.loc[0]["good_prompt"]

"다음 텍스트들은 하나의 강의에 대한 평가들이며, 긍정적인 평가들을 한문장으로 요약해서 {긍정적인평가} 안에 담아서 제공해줘. Translate all answers into Korean language. \n텍스트들: To gain knowledge and new skill for personal interest.\nGood course.\n+ +++++++++++++++++++ + +++++++++++++++++++ +\nPresentation is a little slow. Good examples though.\nGreat content, great app. Bugs - the video player cuts audio when I fast forward sometimes. And, in dark mode, this review box that I’m writing in on the app has white text on a white background, making it extremely hard to see.\nDo you like reading a textbook to learn a course? If so, then this course is for you. If you prefer to have hands on exercises with challenging questions that force you to actually write Ruby programs, this is a waste of time. I made it about halfway through before giving up.\nSo far the course has been a great introduction to the ruby language it offers lots of advice and material on where to get more information\nThe browser player version is much better in resizing the

In [20]:
gptReviews = pd.read_csv("./full_gpt.csv")
gptReviews


,site_lecture_id,gptReviews
0,8082,nce with programming. + +++++++++++++++++++ + ...
1,8319,"장점:\n- 흥미롭고 재미있는 강의\n- 따라하기 쉽고, 깃에 대한 실무 지식을 얻..."
2,8416,. 장점:\n- 초보자에게 좋은 튜토리얼\n- 아이폰 앱 개발에 대한 기초적인 튜토...
3,9463,장점:\n- 명확한 설명\n- 다양한 예제\n- 초보자에게 적합\n\n단점:\n-...
4,9646,Udemy. 강의 내용이 매우 체계적이고 이해하기 쉽다.\n강사님의 영어 발음이 ...
...,...,...
22905,5744038,"장점:\n1. 강의 내용이 알차고, 초보자도 이해하기 쉽다.\n2. 강사가 친절하고..."
22906,5744808,a lot. 장점:\n- 설명이 잘 되어 있다.\n- 유익하다.\n- 시험 문제의 ...
22907,5744908,장점:\n- 강의 내용이 알차고 쉽게 이해할 수 있다.\n- 강의가 잘 구성되어 있...
22908,5745318,장점:\n- 자세한 설명\n- 파이썬으로 통계 분석을 배우기에 가장 좋은 강의\n-...


In [35]:
gptReviews["length"] = gptReviews["gptReviews"].apply(lambda x:len(str(x)))
gptReviews.sort_values("length")

,site_lecture_id,gptReviews,length
16064,4119534,.,1
5520,1534830,.,1
4936,1413802,.,1
7082,1882842,s,1
6527,1738916,t,1
...,...,...,...
9617,2468778,Good explanation\nGood explanation\nGood expla...,1449
12858,3333632,", Holt-Winters, Prophet, Facebook Prophet, Fac...",1517
9810,2511848,rrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrrr...,1536
14753,3791056,\n good course of Docker Certified Associate P...,1584


In [41]:
gptReviews.loc[gptReviews["site_lecture_id"]==3791056]["gptReviews"].item()

'\n good course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certified Associate Practice Exam\ngood course of Docker Certi

In [33]:
v = list(map(str,gptReviews["gptReviews"].values))
maxx  = 0
for i in v:
    maxx = max(len(i),maxx)
maxx

4096

In [27]:
len(str(gptReviews["gptReviews"].values[0]))

384